In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher as SM
import pandas as pd
import numpy as np

C:\Users\Jinwoo Hong\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
fuzz.ratio("this is a test", "this is a test!")
fuzz.token_sort_ratio("Amazon", "aMZon")

91

In [3]:
data = pd.read_csv('dataXData.csv')
company = data['Business Name'].unique()
company = company.astype(str)
data.head()

,Unnamed: 0,Business Name,Industry,City,State
0,0,3M Company,Industrials,NaN,USA
1,1,A.O. Smith Corp,Industrials,NaN,USA
2,2,Abbott Laboratories,Health Care,NaN,USA
3,3,AbbVie Inc.,Health Care,NaN,USA
4,4,Accenture plc,Information Technology,NaN,USA


In [4]:
data.head()

,Unnamed: 0,Business Name,Industry,City,State
0,0,3M Company,Industrials,NaN,USA
1,1,A.O. Smith Corp,Industrials,NaN,USA
2,2,Abbott Laboratories,Health Care,NaN,USA
3,3,AbbVie Inc.,Health Care,NaN,USA
4,4,Accenture plc,Information Technology,NaN,USA


In [5]:
type(company)

numpy.ndarray

In [6]:
print(len(company))

940427


In [7]:
company

array(['3M Company', 'A.O. Smith Corp', 'Abbott Laboratories', ...,
       'DAVID BLACKMORE & ASSOCIATES', 'DESIGNING DIGITALLY, INC.',
       'KGC ENTERPRISES INC'], dtype='<U100')

In [25]:
text=input("Which text do you want to search? ")
country = input("Which country do you want to search? ")
location = input("Where do you want to search? ")

Which text do you want to search? Johnson
Which country do you want to search? NY
Where do you want to search? NY


In [26]:
a=[]
text1=text
text2=location
text3=country
if text3 == 'UK':
    data2 = data[data['State'] == 'UK']
elif text3 == 'Canada':
    data2 = data[data['State'] == 'Canada']
else:
    if(text3 == 'USA'):
        data2 = data[(data['State'] != 'UK') & (data['State'] != 'Canada')]
    else:
        data2 = data[data['State'] == text2]
company = data2['Business Name'].unique()
company = company.astype(str)
for i in range(0,len(company),1):
    search=company[i].lower()
    a.append(fuzz.token_sort_ratio(text1,search))
    # Calculate the edit distance between two strings
ratio=np.array(a)
c=np.where(ratio==ratio.max())
maximum=max(a)
#ind=a.index(maximum)
print("Maximum Fuzz ratio against",text,"is",maximum,"for the brands",company[c])

Maximum Fuzz ratio against Johnson is 74 for the brands ['SIMON, JOHN M' 'JOHNSON, GARY' 'JOHNSON, PAUL' 'JONASON, JOHN'
 'JOHNSON, MARK' 'JOHNSON, JOHN' 'JOHNSON, ROAN' 'JOHNSON, JACK'
 'SALONIA, JOHN' 'Johnson, Cory']


In [10]:
company[c]

array(['3M Company', '3M COMPANY', '3m Company', 'Jm Company',
       'Ms Company', 'Mb Company'], dtype='<U100')

In [11]:
unique_names = []
for d in company[c]:
    lower = d.lower()
    if lower not in unique_names:
        unique_names.append(lower)
        
print(unique_names)

['3m company', 'jm company', 'ms company', 'mb company']
